# Building the SFR2 package

Steps to building the SFR2 package using:
1. DEM of surface
    - This will be used to set the elevation of the river bed and the slope of the river bed along each reach
2. Streamflow info, i.e. gaugings, any other data
    - This will be used to set inflow from Lake Eppalock dam and also to provide observations for model optimisation
3. Polyline of streams
    - This will be used to map the stream to grid and with the combination of the DEM to sort out the ordering from one reach to       the next

### Some assumptions that effect choice of SFR2 parameters:
Flux between surface and subsurface is saturated, i.e. we don't consider the unsaturated zone, or any options parameters associated with this in SFR2.


### Data requirements:
As this package will be required for both a steady state and transient model we will need slightly different setups and parameters.

For the steady state 1a. will be specified as <code>REACHINPUT</code>, but for the transient model, 1a. will be specified as <code>REACHINPUT TRANSROUTE</code>

1b. <code>TABFILES NUMTAB MAXVAL</code>
    Input flows can be specified to segements using tabular data here. This will come from processing the flow data inline with     temporal setup of the model

1c. <code>NSTRM NSS NSFRPAR NPARSEG CONST DLEAK ISTCB1 ISTCB2 {ISFROPT} {NSTRAIL} {ISUZN} {NSFRSETS} {IRTFLG} {NUMTIM} {WEIGHT} {FLWTOL}</code>



The main bits to put together for this outside of the flags and solver options are:
1. reach_data
    - <code>k,i,j,iseg,ireach,rchlen,strtop,slope,strthick,strhc1 </code>
    - <code>k,i,j</code> come from mapping a polyline to the grid
    - <code>iseg</code> segments have to be defined, which we can do for each reach between gauges.
    - <code>rchlen</code> comes from the mapping
    - <code>strtop</code> is the streambed elevation
    - <code>strthick</code> is the thickness of the streambed
    - <code>strhc1</code> is the hydraulic conductivity of the streambed.
2. segment_data
    - <code>nseg,icalc,outseg,iupseg,nstrpts,flow,runoff,etsw,pptsw,roughch,roughbk,cdpth,fdpth,awdth,bwdth,width1,width2</code>
    - nseg
    
3. channel_geometry_data,
4. channel_flow_data,
5. dataset_5,



In [4]:
import os
import subprocess
from osgeo import gdal, osr
import pandas as pd
import numpy as np

## Processing the surface DEM 
This is to translate from the much finer DEM to the grid of the model and rebin the raster pixels using the minimum value

In [5]:
from HydroModelBuilder.GISInterface.GDALInterface import reproject

In [8]:
# Test for raster reprojection
surf_file = r"C:\Workspace\part0075\MDB modelling\ESRI_GRID_raw\ESRI_GRID\sur_1t"
pixel_spacing = 5000
epsg_from = None
epsg_to = 28355

Proj_CS = osr.SpatialReference()
# 28355 is the code for gda94 mga zone 55;
# http://spatialreference.org/ref/epsg/gda94-mga-zone-55/
Proj_CS.ImportFromEPSG(28355)
target_srs = Proj_CS.ExportToWkt()

In [ ]:

# -te xmin ymin xmax ymax # Set bounds
# -tr xres yres #set output file resolution (in target georeferenced units)
# -r resampling_method  # includes min

command = "gdalwarp -t_srs " + target_srs + " -te " + " {0} {1} {2} {3} ".format(xmin, ymin, xmax, ymax) + \
          surf_file + '" "' + sur_min_file + '" -r min'
print(command) 

try:
    subprocess.check_output(command, shell=True)
except subprocess.CalledProcessError as e:
    print(e)
